In [ ]:
from gurobipy import *
import pandas as pd
import numpy as np
import random

Load dataset

In [ ]:
movie_data = pd.read_csv('./ratings_sample_final.csv')
U = movie_data['userId'].to_list()
Ir = movie_data['movieId'].to_list()
R = movie_data['retailerId'].to_list()

constants 

In [ ]:
K = 100000
m = len(set(U))
k = 2
w = 2

Declare and Initialise m1 and m2

In [ ]:
m1 = Model('LUBP1')
m2 = Model('LUBP2')

prep data for u

In [ ]:
x_data = dict(zip(Ir, U))
y_data = dict(zip(R, U))

u = [[0 for _ in range(len(Ir)+1)] for _ in range(len(U)+1)]

for i in Ir:
    for j in U:
        item = i-1
        user = j-1
        if (item, user) in x_data:
            u[item][user] = 1
        else:
            u[item][user] = random.randint(0,1)

u_matrix = np.array(u)

decision variables and constraints for m1

In [ ]:

x_1 = m1.addMVar((501, 501), vtype=GRB.BINARY, name="x_m1") #c3
m1.update

c1 = m1.addConstrs((x_1[:, j].sum() == k for j in U), 'c1')

decision variables and constraints for m2

In [ ]:
x_2 = m2.addMVar((501, 501), vtype=GRB.BINARY, name="x_m2") #c9
y_2 = m2.addMVar((501, 501), vtype=GRB.BINARY, name="y_m2") #c10
m2.update

c4 = m2.addConstrs((x_2[:, j].sum() == k for j in U), 'c4')
c6 = m2.addConstrs((x_2[:, j].sum() <= K*y_2[r,j] for r in R for j in U), 'c6')
c7 = m2.addConstrs((x_2[:, j].sum() <= (1 - K*(1 - y_2[r,j])) for r in R for j in U ), 'c7')
c8 = m2.addConstrs((y_2[:, j].sum() >= w for j in U), 'c8')

initial objective functions for m1 and m2